In [1]:
class state:
    '''input format is as follows:
            numMissionary: Number of missionaries.
            numCannibal: Number of Cannibals.
            boolboat: Whether boat is on the first side or the other.
                    boolBoat = 1, means the boat is on the initial side.
                    boolboat = 0, means the boat is on the ither side.
            gx: number of steps taken to reach current state.
            maxMis: Maximum number of Missionaries.
            maxCan: Maximum number of Cannibals.
            isBFS: Whether search is BFS?
                    1 for Yes.
                    if Yes, 
                        ifAStar becomes 0 automatically.
                    0 for No. 
                    if No,
                        then Branch and Bound is selected. 
            ifAStar: If search method is A Star.
                    1 for yes
                    0 for No
                    if Yes,
                        B&B is selected by default.
                    
            Pruning is done based on Upper Bound, which is calculated as,
                Upper Bound = Maximum number of Missionaries + Maximum number of Cannibals + int(gx/2)
                
            Pruning is done based on the following criteria:
                if Number of Missionaries + Number of Cannibals - 1 <= Upper Bound, 
                    then it is allowed.
                else,
                    removed.'''
    
    
    
    def __init__(self, numMissionary, numCannibal, boolBoat, gx, maxMis, maxCan, isBFS = 1, ifAStar = 0):
        self.numMissionary = numMissionary
        self.numCannibal = numCannibal
        self.numBoat = boolBoat
        self.gx = gx
        self.maxMis = maxMis
        self.maxCan = maxCan
        self.isBFS = isBFS
        self.uBound = self.maxMis + self.maxCan
        if self.isBFS == True:
            self.ifAStar = 0
        else:
            self.ifAStar = ifAStar
            
            
            
    
    def goalTest(self):
        '''Tests if Number of Missionary and Cannibal is equaal to 0 in the initial side.'''
        
        if self.numMissionary ==0 and self.numCannibal == 0:    
            #If number of Missionaries and Cannibals has reached to 0 on initial side.
            
            return "Goal Found! At g(x) = "+str(self.gx)+"!!!"
        else:
            return False
        
        
        
        
    def isValid(self):
        '''Tests if the state entered is valid or not. Test is done for both sides.
        
        CASE I:
                If number of Missionaries on the initial side is 0 on the initial side then, test depends only on
            whether number of missionaries on the other side is more than or equal to number of Cannibals on 
            the other side
        CASE II:
                If there is missionaries on both side of the river. We test if number of missionaries is more or
            equal to number of Cannibals on both sides.
        CASE III:
                If number fo missionaries is zero on the final side, we test only if number of Missionaries is more
            or equal to nummber of Cannibals on the initial side.'''
               
        
        if self.numMissionary == 0:
            return self.maxMis-self.numMissionary>=self.maxCan-self.numCannibal
            #    If number of Missionaries on the initial side is 0 on the initial side then, test depends only on
            # whether number of missionaries on the other side is more than or equal to number of Cannibals on the
            # other side.
            
        elif self.maxMis-self.numMissionary != 0:
            return (self.numCannibal <= self.numMissionary and 
                    self.maxMis-self.numMissionary>=self.maxCan-self.numCannibal)
            #    If there is missionaries on both side of the river. We test if number of missionaries is more or
            # equal to number of Cannibals on both sides.
            
        else:
            return self.numCannibal <= self.numMissionary
            #    If number fo missionaries is zero on the final side, we test only for initial side.
            
            
          
            
    
    def genChild(self):
        '''If the state is Valid, this function generates child of the state.
            Output will be a list of lists.
                Eg:
                    [Child0, Child1, Child2, Child3, ..., Childn]
            Each list in the main list will be child candidate.
            The nested list will contain the following:
                Childx[0]: Number of remaining Missionary.
                Childx[1]: Number of remaining Cannibal
                Childx[2]: Whether boat is on the initial side?
                            True: Initial side.
                            False: Other side.
                Childx[3]: Step number for the child state.
                Childx[4]: Heuristic value for the child state.'''
        
        
        if self.isValid():
        # If the state is valid?
        
            if self.numBoat == False:
            # Boat is on the Final side.
            
                child = []
                # Empty list of child node for current state which is to be populated.
                
                for i in range (3):
                # Number of Missionaries can be 0, 1, 2.
                
                    for j in range (3):
                    # Number of Cannibals can be 0, 1, 2.
                    
                        temp = []
                        # Temporary list used for creating new Child.
                        
                        if i+j<=2 and not(i==0 and j==0) :
                        # Condition 1: number of missionaries and/or cannibals transfered to the 
                        #              intital side cannot be more than 2.
                        # Condition 2: number of Missionaries and cannibals both cannot be zero
                        #              on the boat.
                        
                            temp.append(self.numMissionary+i)
                            # Append number of Missionaries on the initial side after the transaction in the temporary list.
                            
                            temp.append(self.numCannibal+j)
                            # Append number of Cannibals on the initial side after the transaction in the temporary list.
                            
                            temp.append(True)
                            # Append that the boat will be present on the initial side after the transaction.
                            
                            temp.append(self.gx+1)
                            # Append number of steps to take to reach that state. 
                            # Number of steps will get increased by one in the next state.
                            
                            fx = self.gx+temp[0]+temp[1]
                            # fx is the combined value of cost for the state and heuristic value for the child state.
                            
                            if not self.isBFS:
                            # Condition: If the search is Branch and Bound or Astar.
                            
                                if self.ifAStar :
                                # Condition: If the search type is Astar.
                                
                                    temp.append(fx)
                                    # Append the fx value to the temporary list
                                    
                                    if temp[0]<=self.maxMis and temp[1]<=self.maxCan:
                                    # Condition 1: if number of Missionaries is less than or equal to maximum possible
                                    #              number of Missionaries.
                                    # Condition 2: if number of Cannibals is less than or equal to maximum possible
                                    #              number of Cannibals.
                                    
                                        uBound = int((self.gx + 1)/2)+self.uBound
                                        # Estimated Upper Bound for the state generated.
                                        
                                        if fx <=uBound:
                                        # Condition: if the fx value is less than or equal to estimated upper bound.
                                        # Note: This is our Pruning criteria.
                                        
                                            child.append(temp)
                                            # Append the temporary list to the list of child nodes.
                                            
                                else:
                                # If search type is Branch and Bound.
                                
                                    if temp[0]<=self.maxMis and temp[1]<=self.maxCan:
                                    # Condition 1: if number of Missionaries is less than or equal to maximum possible
                                    #              number of Missionaries.
                                    # Condition 2: if number of Cannibals is less than or equal to maximum possible
                                    #              number of Cannibals.
                                    
                                        uBound = int((self.gx + 1)/2)+self.uBound
                                        # Estimated Upper Bound for the state generated.
                                        
                                        if fx <=uBound:
                                        # Condition: if the fx value is less than or equal to estimated upper bound.
                                        # Note: This is our Pruning criteria.
                                        
                                            child.append(temp)
                                            # Append the temporary list to the list of child nodes.
                            else:
                            # If the search type is Breadth First Search.
                            
                                if temp[0]<=self.maxMis and temp[1]<=self.maxCan:
                                # Condition 1: if number of Missionaries is less than or equal to maximum possible
                                #              number of Missionaries.
                                # Condition 2: if number of Cannibals is less than or equal to maximum possible
                                #              number of Cannibals.
                                
                                    child.append(temp)
                                    # Append the temporary list to the list of child nodes.
                return(child)
                        
            elif self.numBoat == True:
            # Boat is on the initial side.
            
                child = []
                # Empty list of child node for current state which is to be populated.
                
                for i in range (3):
                # Number of Missionaries can be 0, 1, 2.
                
                    for j in range (3):
                    # Number of Cannibals can be 0, 1, 2.
                    
                        temp = []
                        # Temporary list used for creating new Child.
                        
                        if i+j<=2 and not(i==0 and j==0):
                        # Condition 1: number of missionaries and/or cannibals transfered to the 
                        #              intital side cannot be more than 2.
                        # Condition 2: number of Missionaries and cannibals both cannot be zero
                        #              on the boat.
                        
                            temp.append(self.numMissionary-i)
                            # Append number of Missionaries on the initial side after the transaction in the temporary list.
                            
                            temp.append(self.numCannibal-j)
                            # Append number of Cannibals on the initial side after the transaction in the temporary list.
                            
                            temp.append(False)
                            # Append that the boat will be absent on the initial side after the transaction.
                            
                            temp.append(self.gx+1)
                            # Append number of steps to take to reach that state. 
                            # Number of steps will get increased by one in the next state.
                            
                            fx = self.gx+temp[0]+temp[1]
                            # fx is the combined value of cost for the state and heuristic value for the child state.
                            
                            if not self.isBFS:
                            # Condition: If the search is Branch and Bound or Astar.
                            
                                if self.ifAStar:
                                # Condition: If the search type is Astar.
                                
                                    temp.append(fx)
                                    # Append the fx value to the temporary list
                                    
                                    if temp[0]>=0 and temp[1]>=0 :
                                    # Condition 1: if number of Missionaries is more than or equal to 0.
                                    # Condition 2: if number of Cannibals is more than or equal to 0.
                                    
                                        uBound = int((self.gx + 1)/2)+self.uBound
                                        # Estimated Upper Bound for the state generated.
                                        
                                        if fx <= uBound:
                                        # Condition: if the fx value is less than or equal to estimated upper bound.
                                        # Note: This is our Pruning criteria.
                                        
                                            child.append(temp)
                                            # Append the temporary list to the list of child nodes.
                                            
                                else:
                                # If search type is Branch and Bound.
                                
                                    if temp[0]>=0 and temp[1]>=0 :
                                    # Condition 1: if number of Missionaries is more than or equal to 0.
                                    # Condition 2: if number of Cannibals is more than or equal to 0.
                                    
                                        uBound = int((self.gx + 1)/2)+self.uBound
                                        # Estimated Upper Bound for the state generated.
                                        
                                        if fx <=uBound:
                                        # Condition: if the fx value is less than or equal to estimated upper bound.
                                        # Note: This is our Pruning criteria.
                                        
                                            child.append(temp)
                                            # Append the temporary list to the list of child nodes.
                                            
                            else:
                            # If the search type is Breadth First Search.
                            
                                if temp[0]>=0 and temp[1]>=0 :
                                # Condition 1: if number of Missionaries is more than or equal to 0.
                                # Condition 2: if number of Cannibals is more than or equal to 0.
                                
                                    child.append(temp)
                                    # Append the temporary list to the list of child nodes.
                                    
                return(child)

In [2]:
class search:
    '''input format is as follows:
            numMis: Number of missionaries.
            numCan: Number of Cannibals.
            searchType: takes string as input
                "bfs": for Breadth First Search.
                "bnb": for Branch and Bound.
                "astar": for A Star.
            withRecord: Whether or not to record a state if it has already appeared.
                "1": for yes.
                "0": for No.
                
                DO NOT PUT withRecord = 0 IF NUMBER OF MISSIONARIES AND CANNIBALS IS LARGE.
                
                Please note that,
                1.  3 Missionaries, 3 Cannibals, and the boat being on the left side (boolBoat = 1)
                        is not same as
                    3 Missionaries, 3 Cannibals, and the boat being on the right side (boolBoat = 0)
                2.  First appearance of any state is not recorded. Only if the state has already been appeared and has gone 
                through goalTest is recorded.
                '''
    def __init__(self, numMis, numCan, searchType = 'bfs', withRecord = 0):
        
        self.numMis = numMis
        # Maximum number of Missionarises possibole.
        
        self.numCan = numCan
        # maximum number of Cannibals Possible.
        
        self.search_queue = [[self.numMis, self.numCan, True, 0, 0, 0]]
        # States to be checked and child node generated are kept in this list.
        
        self.search_record = []
        # Used for back Tracking after the search is completed.
        
        self.search_counter = 0
        # Number of state space explored so far.
        
        self.goal = 0
        # if the goal has been found. Path can be constructed only after this variable becones 1.
        
        self.path = []
        # Empty list that gets populated after the path has been reconstructed.
        
        if withRecord:
            self.recordKeeper = {str(self.numMis)+str(self.numCan)+str(1):1}
        else:
            self.recordKeeper = {}
            
            
        if searchType == 'bfs':
            self.bfs = 1
            self.bnb = 0
            self.astar = 0
        elif searchType == 'bnb':
            self.bfs = 0
            self.bnb = 1
            self.astar = 0
        elif searchType == 'astar':
            self.bfs = 0
            self.bnb = 0
            self.astar = 1
            
            
    def search(self):
        '''Searches the search space as per configuration loaded.'''
        
        while len(self.search_queue)!=0:
        # If search queue is not empty.
        
            if self.astar:
            # If search type is Astar.
            
                self.search_queue.sort(key = lambda x:x[-2])
                # Sort the queue as per the fx value of the available states.
                
            a = self.search_queue.pop(0)
            # Take out the first element from the search queue.
            
            self.search_record.append(a.copy())
            # Append the search state to the search record.
            b = state(a[0], a[1], a[2], a[3], self.numMis, self.numCan, self.bfs, self.astar)
            # Converting the state specification to state.
            
            self.search_counter += 1
            # Increasing number of states tested by 1.
            
            if b.goalTest():
            # If the given state is the goal state.
            
                self.goal = 1
                # Change self.goal to 1.
                
                return('Goal Found After exploring '+str(self.search_counter)+' states.')
                # Return Success.
                
            else:
            # If the given state is not goal state.
            
                if b.genChild():
                # if there is child (valid or invalid) of the state.
                
                    c = b.genChild()
                    # c is the list of child nodes of the given state.
                    
                    for i in c:
                    # For every child state of the given state.
                    
                        i.append(len(self.search_record)-1)
                        # Append the state with the address of its parent state in the search record.
                        
                        if not self.recordKeeper:
                        # If Record of whether or not the state has already been explored in the search is not kept.
                        
                            self.search_queue.append(i)
                            # Append the child state to search queue.
                            
                        if self.recordKeeper:
                        # If Record of whether or not the state has already been explored in the search is being kept.
                        
                            srcStr = str(i[0])+str(i[1])+str(int(i[3]))
                            # Generating search string.
                            
                            try:
                                self.recordKeeper[srcStr] == 1
                                # Check if the search string is there in the record.
                                
                            except:
                            # If the search string is not there.
                            
                                self.search_queue.append(i)
                                # Append the child state to the search queue.
                                
                                self.recordKeeper[srcStr] = 1
                                # Add the search string to the dictionary.
                                
        if len(self.search_queue) == 0:
        # If search string is empty.
        
            return("Goal Not Found")
            # Return Failure.
            
            
    def showPath(self):
        '''Can only be used after successful search.
        Output contains
            a. list of lists.
            b. No. of steps requred to reach goal.
            c. Number of states explored to get the result.
        Each entry contains a state in the form of
            [Number of Missionaries on the initial side, Number of cannibals on the initial side, Whether boat is on the initial side]
            
            Returns:
                01. Path from given state to goal state.
                02. Length of the path.
                03. Number of states explored to reach to the goal state.'''
        
        if self.goal:
        # If goal is found.
        
            c = self.search_record[-1]
            # last element of the search record is the successful state.
            
            while c[3] != 0:
            # Till we reach to the gx = 0 state.
            # Meaning: Till we reach the first state in the search space.
            
                self.path.insert(0, c[:3])
                # Insert the current element as the first element of the path.
                # Only three elements: 1. Number of Missionaries.
                #                      2. number of Cannibals. And
                #                      3. Whether the boat is on the initial side.
                # is required as result. So, only first 3 is chosen.
                
                c = self.search_record[c[-1]]
                # Choose the parent of the current state as current state from search record.
                
            self.path.insert(0,c[:3])
            # Since the loop ended when gx was 0. We append the very last (technically first) state to the list.
            
        return self.path, len(self.path), self.search_counter
        # Return 01. Path from given state to goal state.
        #        02. Length of the path.
        #        03. Number of states explored to reach the goal state.
                

In [3]:
aa = search(3, 3, 'bfs', 0)
aa.search()

'Goal Found After exploring 16469 states.'

In [4]:
aa.showPath()

([[3, 3, True],
  [3, 1, False],
  [3, 2, True],
  [3, 0, False],
  [3, 1, True],
  [1, 1, False],
  [2, 2, True],
  [0, 2, False],
  [0, 3, True],
  [0, 1, False],
  [0, 2, True],
  [0, 0, False]],
 12,
 16469)

In [5]:
ab = search(3, 3, 'bnb', 0)
ab.search()

'Goal Found After exploring 272 states.'

In [6]:
ab.showPath()

([[3, 3, True],
  [3, 1, False],
  [3, 2, True],
  [3, 0, False],
  [3, 1, True],
  [1, 1, False],
  [2, 2, True],
  [0, 2, False],
  [0, 3, True],
  [0, 1, False],
  [0, 2, True],
  [0, 0, False]],
 12,
 272)

In [7]:
ac = search(3,3, 'astar', 0)
ac.search()

'Goal Found After exploring 268 states.'

In [8]:
ac.showPath()

([[3, 3, True],
  [3, 1, False],
  [3, 2, True],
  [3, 0, False],
  [3, 1, True],
  [1, 1, False],
  [2, 2, True],
  [0, 2, False],
  [0, 3, True],
  [0, 1, False],
  [0, 2, True],
  [0, 0, False]],
 12,
 268)

In [9]:
aa = search(4, 3, 'bfs', 0)
aa.search()

'Goal Found After exploring 70483 states.'

In [10]:
aa.showPath()

([[4, 3, True],
  [4, 1, False],
  [4, 2, True],
  [4, 0, False],
  [4, 1, True],
  [2, 1, False],
  [2, 2, True],
  [1, 1, False],
  [2, 1, True],
  [1, 0, False],
  [1, 1, True],
  [0, 0, False]],
 12,
 70483)

In [11]:
ab = search(4, 3, 'bnb', 0)
ab.search()

'Goal Found After exploring 1706 states.'

In [12]:
ab.showPath()

([[4, 3, True],
  [4, 1, False],
  [4, 2, True],
  [4, 0, False],
  [4, 1, True],
  [2, 1, False],
  [2, 2, True],
  [1, 1, False],
  [2, 1, True],
  [1, 0, False],
  [1, 1, True],
  [0, 0, False]],
 12,
 1706)

In [13]:
ac = search(4,3, 'astar', 0)
ac.search()

'Goal Found After exploring 1096 states.'

In [14]:
ac.showPath()

([[4, 3, True],
  [4, 1, False],
  [4, 2, True],
  [4, 0, False],
  [4, 1, True],
  [2, 1, False],
  [2, 2, True],
  [1, 1, False],
  [2, 1, True],
  [1, 0, False],
  [1, 1, True],
  [0, 0, False]],
 12,
 1096)